# Problem

Classical machine learning classification uses cross entropy loss. It works well when classes are well-seperated, but struggles in real-world scenarios where:
1. High intra-class variance: Samples from the same class look very different(e.g; the same person photographed from different angles,lightening and angles).
2. Low inter-class variance: Samples from different classes look similar(e.g., two different people who happen to look alike).

Applications like face recognition, fingerprint matching, image retrieval, and  person re-identification all face this challenge.

# Cross-Entropy Loss: Complete Derivation

## Part 1: The Classification Setup

Consider a neural network for classification with $C$ classes.

Given an input $x$, the network produces a **logit** (raw score) for each class:

$$z_j = W_j^T f + b_j \quad \text{for class } j \in \{1, 2, \ldots, C\}$$

where:
- $f$ is the feature vector (output of the network before the final layer)
- $W_j$ is the weight vector for class $j$
- $b_j$ is the bias for class $j$

## Part 2: The Problem — Converting Scores to Probabilities

We need to convert logits $z_1, z_2, \ldots, z_C$ into probabilities.

**Requirements for valid probabilities:**
1. Each probability must be non-negative: $P(y=j|x) \geq 0$
2. Probabilities must sum to 1: $\sum_{j=1}^{C} P(y=j|x) = 1$

**Why can't we just normalize the logits directly?**

$$P(y=j|x) = \frac{z_j}{\sum_k z_k} \quad \text{❌ WRONG}$$

**Problem**: If $z_j = -5$, we get a negative probability, which is invalid.

## Part 3: The Softmax Function

### Step 1: Make Everything Positive

Apply the exponential function to each logit:

$$e^{z_j} > 0 \quad \text{for any } z_j \in \mathbb{R}$$

This maps any real number to a positive number.

### Step 2: Normalize

Divide by the sum to ensure probabilities sum to 1:

$$\boxed{P(y=j|x) = \frac{e^{z_j}}{\sum_{k=1}^{C} e^{z_k}}}$$

This is the **softmax function**.

### Why Exponential Specifically?

The exponential function has a special property. If we compute the log-odds between two classes:

$$\log \frac{P(y=j|x)}{P(y=k|x)} = \log \frac{e^{z_j}}{e^{z_k}} = z_j - z_k$$

The **difference in logits equals the log-odds**. This gives a clean, interpretable relationship.

### Properties of Softmax

| Property | Explanation |
|----------|-------------|
| Outputs in $(0, 1)$ | Each output is a valid probability |
| Sum to 1 | $\sum_j P(y=j\|x) = 1$ |
| Preserves ranking | Higher logit → higher probability |
| Differentiable | Enables gradient-based optimization |


## Part 4: Maximum Likelihood Estimation

### The Goal

We have training data: $\{(x_1, y_1), (x_2, y_2), \ldots, (x_N, y_N)\}$

We want to find parameters $\theta = \{W_j, b_j\}$ that best explain this data.

### The Likelihood Function

**Key question**: Given parameters $\theta$, what is the probability that our model would produce the observed labels?

For a single sample $(x_i, y_i)$:

$$L_i(\theta) = P(y_i | x_i; \theta)$$

This is the probability our model assigns to the **true class**.

### Example

Suppose for sample $i$:
- True class: "cat"
- Model outputs: $P(\text{cat}) = 0.8$, $P(\text{dog}) = 0.15$, $P(\text{bird}) = 0.05$

Then $L_i(\theta) = 0.8$

### Likelihood for the Entire Dataset

Assuming samples are **independent**, the joint probability is the product:

$$L(\theta) = \prod_{i=1}^{N} P(y_i | x_i; \theta)$$

**Why multiply?** For independent events A and B:

$$P(A \text{ and } B) = P(A) \times P(B)$$

### Maximum Likelihood Estimation (MLE)

**Principle**: Choose parameters that maximize the likelihood.

$$\hat{\theta} = \arg\max_{\theta} \prod_{i=1}^{N} P(y_i | x_i; \theta)$$


## Part 5: Log-Likelihood

### The Numerical Problem

With $N = 10000$ samples:

$$L(\theta) = 0.8 \times 0.7 \times 0.9 \times \ldots \approx 10^{-500}$$

This causes **numerical underflow** — computers can't represent such small numbers.

### The Solution: Take the Logarithm

Since $\log$ is a **monotonically increasing function**:

$$\arg\max_\theta L(\theta) = \arg\max_\theta \log L(\theta)$$

The parameters that maximize $L$ also maximize $\log L$.

### Log of a Product Becomes a Sum

$$\log L(\theta) = \log \prod_{i=1}^{N} P(y_i|x_i) = \sum_{i=1}^{N} \log P(y_i|x_i)$$

Now we're adding log-probabilities instead of multiplying probabilities. Much more stable!


## Part 6: From Log-Likelihood to Cross-Entropy Loss

### Maximizing vs. Minimizing

In machine learning, we typically **minimize a loss function** rather than maximize likelihood.

Simple fix — take the negative:

$$\text{Negative Log-Likelihood} = -\log L(\theta) = -\sum_{i=1}^{N} \log P(y_i|x_i)$$

Minimizing this is equivalent to maximizing the likelihood.

### Cross-Entropy Loss (Single Sample)

For one sample with true class $y$:

$$L_{CE} = -\log P(y|x)$$

Substituting the softmax:

$$L_{CE} = -\log \frac{e^{z_y}}{\sum_{k=1}^{C} e^{z_k}}$$

### Expanding the Logarithm

Using $\log \frac{a}{b} = \log a - \log b$:

$$L_{CE} = -\left( \log e^{z_y} - \log \sum_{k=1}^{C} e^{z_k} \right)$$

$$\boxed{L_{CE} = -z_y + \log \sum_{k=1}^{C} e^{z_k}}$$

**Interpretation**:
- $-z_y$: We want the logit of the true class to be **high** (so this term is small)
- $\log \sum_k e^{z_k}$: We want other logits to be **low** (so this term is small)


## Part 7: General Form with One-Hot Encoding

### One-Hot Representation

Instead of representing the true class as a single integer $y$, we use a **one-hot vector**:

$$\mathbf{y} = [y_1, y_2, \ldots, y_C]$$

where:

$$y_j = \begin{cases} 1 & \text{if } j \text{ is the true class} \\ 0 & \text{otherwise} \end{cases}$$

### Example

For 3 classes (cat, dog, bird) where the true class is "dog":

$$\mathbf{y} = [0, 1, 0]$$

### Cross-Entropy with One-Hot Encoding

$$L_{CE} = -\sum_{j=1}^{C} y_j \log P(y=j|x)$$

Since $y_j = 0$ for all classes except the true class, this simplifies to:

$$L_{CE} = -\log P(y=\text{true class}|x)$$

which matches our earlier formula.

### Full Formula

$$\boxed{L_{CE} = -\sum_{j=1}^{C} y_j \log \frac{e^{z_j}}{\sum_{k=1}^{C} e^{z_k}}}$$


## Part 8: Connection to Information Theory

### What is Entropy?

For a probability distribution $p$, **entropy** measures uncertainty:

$$H(p) = -\sum_j p_j \log p_j$$

### What is Cross-Entropy?

**Cross-entropy** between true distribution $p$ and predicted distribution $q$:

$$H(p, q) = -\sum_j p_j \log q_j$$

In classification:
- $p$ = one-hot vector (true distribution, all mass on one class)
- $q$ = softmax output (predicted distribution)

So our loss is literally the cross-entropy between true and predicted distributions!

### Relationship to KL Divergence

$$H(p, q) = H(p) + D_{KL}(p \| q)$$

Since $H(p) = 0$ for one-hot vectors:

$$L_{CE} = D_{KL}(p \| q)$$

Minimizing cross-entropy = minimizing KL divergence from true to predicted distribution.


## Gradient of Cross-Entropy Loss

For backpropagation, we need $\frac{\partial L_{CE}}{\partial z_j}$.

### Result

$$\frac{\partial L_{CE}}{\partial z_j} = P(y=j|x) - y_j = \text{softmax}(z_j) - y_j$$

### Interpretation

| True class? | $y_j$ | $P(y=j\|x)$ | Gradient |
|-------------|-------|-------------|----------|
| Yes | 1 | e.g., 0.8 | $0.8 - 1 = -0.2$ (push up) |
| No | 0 | e.g., 0.15 | $0.15 - 0 = 0.15$ (push down) |

The gradient:
- **Increases** the logit of the true class
- **Decreases** the logits of incorrect classes
- Magnitude depends on how wrong the prediction is


##  Summary

| Step | What We Do | Why |
|------|------------|-----|
| 1. Compute logits | $z_j = W_j^T f + b_j$ | Linear transformation of features |
| 2. Apply softmax | $P(j\|x) = \frac{e^{z_j}}{\sum_k e^{z_k}}$ | Convert to valid probabilities |
| 3. Compute loss | $L = -\log P(\text{true class}\|x)$ | Negative log-likelihood |
| 4. Backpropagate | $\nabla_z L = \text{softmax}(z) - y$ | Update parameters |

### The Complete Pipeline

```
Input x
    ↓
Neural Network → Feature f
    ↓
Linear Layer → Logits z = Wf + b
    ↓
Softmax → Probabilities P(y|x)
    ↓
Cross-Entropy Loss with true label y
    ↓
Backpropagation to update W, b
```

## Key Takeaways

1. **Softmax** converts unconstrained logits to valid probabilities
2. **Cross-entropy loss** comes from maximum likelihood estimation
3. **Log transformation** converts products to sums for numerical stability
4. **Negative sign** converts maximization to minimization
5. The gradient has an elegant form: $\text{prediction} - \text{target}$